In [3]:
import billboard
import requests
from pprint import pprint
import re
from datetime import datetime, timedelta

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

## tests

In [2]:
date = '1995-05-15'

In [3]:
chart = billboard.ChartData('hot-100', date='1995-05-15')
chart[0:3]

[billboard.ChartEntry(title='This Is How We Do It', artist='Montell Jordan'),
 billboard.ChartEntry(title='Have You Ever Really Loved A Woman?', artist='Bryan Adams'),
 billboard.ChartEntry(title='Freak Like Me', artist='Adina Howard')]

In [18]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="4ea6e458711041f38a00cb6c72d32d8f",
                                                           client_secret="a69b5cccf41c434da8a2d1d3aa839e44"))

results = sp.search(q='This Is How We Do It Montell Jordan', type="track", limit=1)
for idx, track in enumerate(results['tracks']['items']):
    obj = { 'track':track['uri'], 'popularity':track['popularity'] }
    pprint(obj)

{'popularity': 68, 'track': 'spotify:track:6uQKuonTU8VKBz5SHZuQXD'}


## work

In [4]:
# define spotify auth

spotify_auth = {
    'client_id':'4ea6e458711041f38a00cb6c72d32d8f',
    'client_secret': 'a69b5cccf41c434da8a2d1d3aa839e44'
}

In [5]:
def get_spotify_track(title, artist, spotify_auth):
    '''
    a function that takes a track and artist as string arguments
    alongside a spotify_auth object with client_id ans client_secret
    and returns the spotify track:uri and track:popularity
    '''
    
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
        client_id=spotify_auth['client_id'], 
        client_secret=spotify_auth['client_secret']
    ))
    
    results = sp.search(q=f'{title} {artist}', type="track", limit=1)
    # this loop feels unecessary
    for idx, track in enumerate(results['tracks']['items']):
        obj = { 'uri':track['uri'], 'popularity':track['popularity'] }
        return obj

In [6]:
def get_conception_date(date):
    '''
    a function that takes a date_str yyyy-mm-dd
    and returns a date_str yyyy-mm-dd
    for the date 270 days prior
    '''
    
    date = datetime.strptime(date, "%Y-%m-%d").date()
    conception_date = date - timedelta(days=270)
    date_str = datetime.strftime(conception_date, "%Y-%m-%d")
    return date_str

In [7]:
def get_conception_date_tracks(date, mixtape_len):
    '''
    a function to return a cc2000 object for site utlity
    
    function takes:
        - a date_str yyyy-mm-dd
        - a mixtape_len
        
    and returns the indexed billboard top-100 track playing 270 days before the input date
    '''
    
    # validate date
    if not re.search("\d\d\d\d-\d\d-\d\d", date):
        print('incorrect date format')
        return
    
    # get conception date
    conception_date = get_conception_date(date)
    
    # get hot 100 chart for date
    try:
        chart = billboard.ChartData('hot-100', date=conception_date)
        # chart = chart[0:mixtape_len]

    except:
        print('chart error')
        return
    
    if not chart:
        return # return if no chart object
    
    tracks = []
    i = 0
    num_tracks = 0
    
    while num_tracks < mixtape_len:
        # print(num_tracks)
        try:
            track = {}
            track['title'] = chart[i].title
            track['artist'] = chart[i].artist

            track_spotify_data = get_spotify_track(chart[i].title, chart[i].artist, spotify_auth)
            track['spotify_uri'] = track_spotify_data['uri']
            track['popularity'] = track_spotify_data['popularity']

            tracks.append(track)
            i+=1
            num_tracks += 1
            
        except:
            i+=1
            pass # pass if spotify can't find track, still build list of 10/mixtape_len tracks
    
    return tracks

In [8]:
def create_conception_track_day_object(tracks):
    '''
    takes a list of track objects (artist, popularity, sporitfy_uri, title)
    and returns an object with keys relating to populatriy metric (for spicy mettric match)
    '''
    
    if not tracks:
        return {'error':'no tracks'}
    
    sorted_tracks = sorted(tracks, key=lambda x: x['popularity'])
    
    day_object = {}
    i=1
    
    for item in sorted_tracks:
        day_object[i] = item
        i += 1
    
    return day_object

In [32]:
def make_day_object(date, mixtape_len):
    '''
    a function that takes a date and a mixtape length
    and returns the ordered cc2000 day object with track and spotify data
    tracks are ordered by 1-10 (or mixtape_len) to match cc2000 spicy rank
    '''
    
    tracks = get_conception_date_tracks(date, mixtape_len)
    day_object = create_conception_track_day_object(tracks)
    
    return day_object

In [29]:
def make_cc2000_data():
    '''
    makes the output dictionary for cc2000
    starts from current date and works backwards to 1958-08-04
    returns object with dates yyyy-mm-dd as keys
    '''
    
    cc2000_data = {}
    
    end_date_str = '1958-08-04' # this is the day the first billboard hot 100 was released
    end_date_str = '2021-12-09' # this is a closer test date
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d").date()
    
    date = datetime.today().date()
    
    while date >= end_date: # end once billboard runs out of data
   
        print(date)
        date_str = datetime.strftime(date, "%Y-%m-%d")
        day_object = make_day_object(date_str, 10)
        
        cc2000_data[date_str] = day_object
        
        # decrease by one day
        date -= timedelta(days = 1)
    
    return cc2000_data

In [30]:
data = make_cc2000_data()

2021-12-12
2021-12-11
2021-12-10
2021-12-09


In [31]:
data

{'2021-12-12': {1: {'title': 'Save Your Tears',
   'artist': 'The Weeknd & Ariana Grande',
   'spotify_uri': 'spotify:track:1EGf8RqJ3pGqVzumPYehuH',
   'popularity': 14},
  2: {'title': "What's Next",
   'artist': 'Drake',
   'spotify_uri': 'spotify:track:3mDFLytDotXo2p0rvfGbkA',
   'popularity': 62},
  3: {'title': 'Up',
   'artist': 'Cardi B',
   'spotify_uri': 'spotify:track:1XXimziG1uhM0eDNCZCrUl',
   'popularity': 80},
  4: {'title': 'Go Crazy',
   'artist': 'Chris Brown & Young Thug',
   'spotify_uri': 'spotify:track:1IIKrJVP1C9N7iPtG6eOsK',
   'popularity': 81},
  5: {'title': 'What You Know Bout Love',
   'artist': 'Pop Smoke',
   'spotify_uri': 'spotify:track:1tkg4EHVoqnhR6iFEXb60y',
   'popularity': 85},
  6: {'title': 'Levitating',
   'artist': 'Dua Lipa',
   'spotify_uri': 'spotify:track:5nujrmhLynf4yMoMtj8AQF',
   'popularity': 88},
  7: {'title': 'Positions',
   'artist': 'Ariana Grande',
   'spotify_uri': 'spotify:track:35mvY5S1H3J2QZyna3TFe0',
   'popularity': 88},
  8: